In [1]:
# imports
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.utils import shuffle
from keras.layers import LSTM
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

# Import Keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.regularizers import l2
from time import time
import matplotlib as plt

In [2]:
path_dir = os.path.dirname("")

# read csv file
df = pd.read_csv(os.path.join(path_dir, 'train_ts.csv'))

In [3]:
path_dir = os.path.dirname("")

# read csv file
df = pd.read_csv(os.path.join(path_dir, 'train_ts.csv'))

# shuffle data
df = shuffle(df)

df['timeseries'] = df['timeseries'].apply(lambda x : x.replace("[","").replace("]","").replace(" ",""))
df['pheno_metrics'] = df['pheno_metrics'].apply(lambda x : x.replace("[","").replace("]","").replace(" ",""))

df['timeseries_pheno_metrics'] = df['timeseries'] +','+df['pheno_metrics']
df['timeseries_pheno_metrics'] = df['timeseries_pheno_metrics'].apply(lambda x : x.split(','))
df['timeseries_pheno_metrics'] = df['timeseries_pheno_metrics'].apply(lambda x : [float(i) for i in x])
df['timeseries_pheno_metrics'] = df['timeseries_pheno_metrics'].apply(lambda x : np.array(x, dtype=np.float32) )

X_train = df['timeseries_pheno_metrics'] 
y_train = df["label_id"]

In [4]:
X_train.shape, y_train.shape

((862,), (862,))

In [5]:
T = 23

samples = 129

train_timeseries = X_train[samples:]
train_labels = y_train[samples:]

test_timeseries = X_train[:samples]
test_labels = y_train[:samples]

num_classes = len(np.unique(y_train))

print(num_classes)

print(train_timeseries.shape, train_labels.shape, test_timeseries.shape, test_labels.shape)

7
(733,) (733,) (129,) (129,)


In [6]:
X_train, y_train = [], []
for i in range(train_labels.shape[0] - (T-1)):
    X_train.append(np.expand_dims(train_timeseries.iloc[i:i+T].values, axis=1))
    y_train.append(train_labels.iloc[i + (T-1)])
X_train, y_train = np.array(X_train), np.array(y_train).reshape(-1,1)
print(f'Train data dimensions: {X_train.shape}, {y_train.shape}')

X_test, y_test = [], []
for i in range(test_labels.shape[0]):
    X_test.append(np.expand_dims(train_timeseries.iloc[i:i+T].values, axis=1))
    y_test.append(test_labels.iloc[i])
X_test, y_test = np.array(X_test), np.array(y_test).reshape(-1,1)  

print(f'Test data dimensions: {X_test.shape}, {y_test.shape}')

Train data dimensions: (711, 23, 1), (711, 1)
Test data dimensions: (129, 23, 1), (129, 1)


In [12]:
X_train[0][0][0]

array([ 9.0330000e+03,  8.9820000e+03,  8.9770000e+03,  9.0570000e+03,
        9.0360000e+03,  9.0400000e+03,  8.9040000e+03,  9.0230000e+03,
        9.0020000e+03,  8.9170000e+03,  8.8590000e+03,  8.9370000e+03,
        8.9270000e+03,  8.8930000e+03,  8.7330000e+03,  8.2850000e+03,
        8.7900000e+03,  8.8860000e+03, -9.9990000e+03, -9.9990000e+03,
        9.0650000e+03,  8.9520000e+03,  9.3430000e+03,  9.1391426e+03,
        4.0295834e+02,            nan,  0.0000000e+00,  0.0000000e+00,
        1.5845500e+05,  3.7843300e+05,  1.9342000e+04, -3.2800000e+02,
        4.8000000e+01, -9.9990000e+03,  3.0500000e+02,  9.3430000e+03,
        3.5300000e+02, -9.9990000e+03,  2.8900000e+02,            nan,
        0.0000000e+00], dtype=float32)

In [ ]:
LAYERS = [8, 8, 8, 1]                # number of units in hidden and output layers
M_TRAIN = X_train.shape[0]           # number of training examples (2D)
M_TEST = X_test.shape[0]             # number of test examples (2D),full=X_test.shape[0]
N = X_train.shape[2]                 # number of features
BATCH = M_TRAIN                          # batch size
EPOCH = 50                           # number of epochs
LR = 5e-2                            # learning rate of the gradient descent
LAMBD = 3e-2                         # lambda in L2 regularizaion
DP = 0.0                             # dropout rate
RDP = 0.0                            # recurrent dropout rate
print(f'layers={LAYERS}, train_examples={M_TRAIN}, test_examples={M_TEST}')
print(f'batch = {BATCH}, timesteps = {T}, features = {N}, epochs = {EPOCH}')
print(f'lr = {LR}, lambda = {LAMBD}, dropout = {DP}, recurr_dropout = {RDP}')

# Build the Model
model = Sequential()
model.add(LSTM(input_shape=(T, N), units=LAYERS[0],
               activation='tanh', recurrent_activation='hard_sigmoid',
               kernel_regularizer=l2(LAMBD), recurrent_regularizer=l2(LAMBD),
               dropout=DP, recurrent_dropout=RDP,
               return_sequences=True, return_state=False,
               stateful=False, unroll=False
              ))
model.add(BatchNormalization())
model.add(LSTM(units=LAYERS[1],
               activation='tanh', recurrent_activation='hard_sigmoid',
               kernel_regularizer=l2(LAMBD), recurrent_regularizer=l2(LAMBD),
               dropout=DP, recurrent_dropout=RDP,
               return_sequences=True, return_state=False,
               stateful=False, unroll=False
              ))
model.add(BatchNormalization())
model.add(LSTM(units=LAYERS[2],
               activation='tanh', recurrent_activation='hard_sigmoid',
               kernel_regularizer=l2(LAMBD), recurrent_regularizer=l2(LAMBD),
               dropout=DP, recurrent_dropout=RDP,
               return_sequences=False, return_state=False,
               stateful=False, unroll=False
              ))
model.add(BatchNormalization())
model.add(Dense(units=LAYERS[3], activation='sigmoid'))

# Compile the model with Adam optimizer
model.compile(loss='binary_crossentropy',
              metrics=['accuracy'],
              optimizer=Adam(learning_rate=LR))
print(model.summary())

# Define a learning rate decay method:
lr_decay = ReduceLROnPlateau(monitor='loss', 
                             patience=1, verbose=0, 
                             factor=0.5, min_learning_rate=1e-8)
# Define Early Stopping:
early_stop = EarlyStopping(monitor='val_acc', min_delta=0, 
                           patience=30, verbose=1, mode='auto',
                           baseline=0, restore_best_weights=True)
# Train the model. 
# The dataset is small for NN - let's use test_data for validation
start = time()
History = model.fit(X_train, y_train,
                    epochs=EPOCH,
                    batch_size=BATCH,
                    validation_split=0.0,
                    validation_data=(X_test[:M_TEST], y_test[:M_TEST]),
                    shuffle=True,verbose=0,
                    callbacks=[lr_decay, early_stop])
print('-'*65)
print(f'Training was completed in {time() - start:.2f} secs')
print('-'*65)